In [1]:
import urllib.request
from bs4 import BeautifulSoup

In [2]:
link = 'https://www.ed.ac.uk/studying/postgraduate/degrees/index.php&edition=2023?r=site%2Fsearch&pgSearch=&yt0=&moa=a'

In [3]:
website_html = urllib.request.urlopen(link).read()

In [4]:
soup = BeautifulSoup(website_html,'lxml')

In [5]:
#soup.prettify()

In [6]:
courses = soup.find_all('a',class_='list-group-item')

In [7]:
len(courses)

526

In [80]:
PG_dict = {}
for course in courses:
    domain1 = 'https://www.ed.ac.uk'
    domain2 = course['href']
    domain = domain1+domain2
    id = domain.split('=')[-1]
    name_of_course = course.text
    #first 5 are space
    #print(name_of_course.split(' '))
    o_or_no = name_of_course.split(' ')[0:1]
    if '\nonline\n' in o_or_no:
        o_or_no = 'Online'
    else:
        o_or_no = 'Offline'
    ls = name_of_course.split(' ')[1:-1]
    #print(ls)
    sep = name_of_course.split(' ')[1:-1].index('\n')
    course_name = ' '.join(ls[3:sep-1])
    degree_types = ' '.join(ls[sep+4:-1])
    #print(len(name_of_course))
    #print('*'*20)
    website_link = urllib.request.urlopen(domain).read()
    soup = BeautifulSoup(website_link,'lxml')
    details = soup.find('div',class_='col-xs 12')
    #print(details.ul.text.split('\n'))
    #print(len(details.ul.text.split('\n')))
    School_index = details.ul.text.split('\n').index('School: ')
    College_index = details.ul.text.split('\n').index('College: ')
    #print(School_index,College_index)
    School = ' '.join(details.ul.text.split('\n')[School_index+2:College_index]).strip()
    College = ' '.join(details.ul.text.split('\n')[College_index+1:]).strip()
    programm_code = soup.find_all('table',class_='table table-striped')
    #print(programm_code)
    try:
        programm_code = programm_code[1].tbody.a['href'].split('=')[-1]
    except:
        programm_code = 'No Program Code'
    PG_dict[id] = [course_name,programm_code,degree_types,o_or_no,domain,School,College]

In [81]:
len(PG_dict)

526

In [83]:
import pandas as pd

In [87]:
PG_df = pd.DataFrame(PG_dict).T

In [89]:
PG_df['ID'] = PG_df.index

In [92]:
PG_df.reset_index(drop=True,inplace=True)

In [94]:
PG_df.rename({0:'Program',1:'Program_Code',2:'Degrees',3:'Mode_of_study',4:'Link',5:'School',6:'College'},axis=1,inplace=True)

In [96]:
PG_df.head()

,Program,Program_Code,Degrees,Mode_of_study,Link,School,College,ID
0,Accounting,PRPHDACCOU1F,PhD,Offline,https://www.ed.ac.uk/studying/postgraduate/deg...,Business School,"Arts, Humanities & Social Sciences",1028
1,Accounting and Finance,PTMSCACFIN1F,MSc,Offline,https://www.ed.ac.uk/studying/postgraduate/deg...,Business School,"Arts, Humanities & Social Sciences",416
2,Acoustics and Music Technology,PTMSCACMUT1F,MSc,Offline,https://www.ed.ac.uk/studying/postgraduate/deg...,Edinburgh College of Art,"Arts, Humanities & Social Sciences",478
3,Advanced Care,PRPHDADVCA1F,PhD with Integrated Study,Offline,https://www.ed.ac.uk/studying/postgraduate/deg...,Engineering,Science & Engineering,1048
4,Advanced Chemical Engineering,PTMSCADVCE1F,MSc,Offline,https://www.ed.ac.uk/studying/postgraduate/deg...,Engineering,Science & Engineering,913


In [97]:
PG_df.to_csv('UOE_PG_Courses.csv')